# Hybrid Distillation Column Model

This notebook implements a hybrid model to predict product compositions (MoleFractionTX and MoleFractionHX) based on operating conditions of a distillation column.

The hybrid model combines:
1. **First Principles Model**: Based on fundamental chemical engineering principles
2. **Machine Learning Model**: Random Forest trained on historical data

## Model Overview

- **Target Variables**: MoleFractionTX and MoleFractionHX
- **Input Features**: Operating conditions from the distillation column
- **Hybrid Approach**: Weighted combination of first principles and ML predictions

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.metrics import mean_squared_error, r2_score
import ipywidgets as widgets
from IPython.display import display, clear_output

# Import our custom models
from first_principles_model import first_principles_distillation_model
from hybrid_model import hybrid_distillation_model

## 1. Data Exploration

In [ ]:
# Load the dataset
df = pd.read_excel('/home/ubuntu/upload/DistillationColumnDataset.xlsx')

print("Dataset Shape:", df.shape)
print("\nColumn Names:")
for i, col in enumerate(df.columns):
    print(f"{i+1}. {col}")

print("\nFirst 5 rows:")
display(df.head())

In [ ]:
# Statistical summary
print("Statistical Summary:")
display(df.describe())

In [ ]:
# Visualize target variables
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# MoleFractionTX distribution
axes[0].hist(df['MoleFractionTX'], bins=30, alpha=0.7, color='blue')
axes[0].set_title('Distribution of MoleFractionTX')
axes[0].set_xlabel('MoleFractionTX')
axes[0].set_ylabel('Frequency')

# MoleFractionHX distribution
axes[1].hist(df['MoleFractionHX'], bins=30, alpha=0.7, color='red')
axes[1].set_title('Distribution of MoleFractionHX')
axes[1].set_xlabel('MoleFractionHX')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 2. Model Performance Evaluation

In [ ]:
# Load the trained Random Forest model
rf_model = joblib.load('/home/ubuntu/random_forest_model.joblib')

# Prepare features for evaluation
X = df.drop(['Time', 'MoleFractionTX', 'MoleFractionHX'], axis=1)
y_true = df[['MoleFractionTX', 'MoleFractionHX']]

# Random Forest predictions
rf_predictions = rf_model.predict(X)

# Calculate metrics for Random Forest
rf_mse = mean_squared_error(y_true, rf_predictions)
rf_r2 = r2_score(y_true, rf_predictions)

print(f"Random Forest Model Performance:")
print(f"MSE: {rf_mse:.2e}")
print(f"R²: {rf_r2:.4f}")

In [ ]:
# Visualize Random Forest predictions vs actual
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# MoleFractionTX
axes[0].scatter(y_true['MoleFractionTX'], rf_predictions[:, 0], alpha=0.6)
axes[0].plot([y_true['MoleFractionTX'].min(), y_true['MoleFractionTX'].max()], 
             [y_true['MoleFractionTX'].min(), y_true['MoleFractionTX'].max()], 'r--', lw=2)
axes[0].set_xlabel('Actual MoleFractionTX')
axes[0].set_ylabel('Predicted MoleFractionTX')
axes[0].set_title('Random Forest: MoleFractionTX Predictions')

# MoleFractionHX
axes[1].scatter(y_true['MoleFractionHX'], rf_predictions[:, 1], alpha=0.6)
axes[1].plot([y_true['MoleFractionHX'].min(), y_true['MoleFractionHX'].max()], 
             [y_true['MoleFractionHX'].min(), y_true['MoleFractionHX'].max()], 'r--', lw=2)
axes[1].set_xlabel('Actual MoleFractionHX')
axes[1].set_ylabel('Predicted MoleFractionHX')
axes[1].set_title('Random Forest: MoleFractionHX Predictions')

plt.tight_layout()
plt.show()

## 3. Interactive Hybrid Model Prediction

In [ ]:
# Create interactive widgets for input parameters
def create_input_widgets():
    # Get feature ranges from the dataset for realistic slider ranges
    feature_ranges = {}
    for col in X.columns:
        feature_ranges[col] = (df[col].min(), df[col].max(), df[col].mean())
    
    widgets_dict = {}
    
    # Create sliders for each feature
    widgets_dict['liquid_condenser'] = widgets.FloatSlider(
        value=feature_ranges['Liquid Percentage in Condenser'][2],
        min=feature_ranges['Liquid Percentage in Condenser'][0],
        max=feature_ranges['Liquid Percentage in Condenser'][1],
        step=0.1,
        description='Liquid % Condenser:',
        style={'description_width': 'initial'}
    )
    
    widgets_dict['condenser_pressure'] = widgets.FloatSlider(
        value=feature_ranges['Condenser Pressure'][2],
        min=feature_ranges['Condenser Pressure'][0],
        max=feature_ranges['Condenser Pressure'][1],
        step=0.1,
        description='Condenser Pressure:',
        style={'description_width': 'initial'}
    )
    
    widgets_dict['liquid_reboiler'] = widgets.FloatSlider(
        value=feature_ranges['Liquid Percentage in Reboiler'][2],
        min=feature_ranges['Liquid Percentage in Reboiler'][0],
        max=feature_ranges['Liquid Percentage in Reboiler'][1],
        step=0.1,
        description='Liquid % Reboiler:',
        style={'description_width': 'initial'}
    )
    
    widgets_dict['feed_flow'] = widgets.FloatSlider(
        value=feature_ranges['Mass Flow Rate in Feed Flow'][2],
        min=feature_ranges['Mass Flow Rate in Feed Flow'][0],
        max=feature_ranges['Mass Flow Rate in Feed Flow'][1],
        step=1.0,
        description='Feed Flow Rate:',
        style={'description_width': 'initial'}
    )
    
    widgets_dict['top_outlet_flow'] = widgets.FloatSlider(
        value=feature_ranges['Mass Flow Rate in Top Outlet Stream'][2],
        min=feature_ranges['Mass Flow Rate in Top Outlet Stream'][0],
        max=feature_ranges['Mass Flow Rate in Top Outlet Stream'][1],
        step=1.0,
        description='Top Outlet Flow:',
        style={'description_width': 'initial'}
    )
    
    widgets_dict['net_mass_flow'] = widgets.FloatSlider(
        value=feature_ranges['Net Mass Flow in main tower'][2],
        min=feature_ranges['Net Mass Flow in main tower'][0],
        max=feature_ranges['Net Mass Flow in main tower'][1],
        step=1.0,
        description='Net Mass Flow:',
        style={'description_width': 'initial'}
    )
    
    widgets_dict['hx_reboiler'] = widgets.FloatSlider(
        value=feature_ranges['Mole Fraction HX at reboiler'][2],
        min=feature_ranges['Mole Fraction HX at reboiler'][0],
        max=feature_ranges['Mole Fraction HX at reboiler'][1],
        step=0.001,
        description='HX Mole Fraction Reboiler:',
        style={'description_width': 'initial'}
    )
    
    widgets_dict['hx_top_outlet'] = widgets.FloatSlider(
        value=feature_ranges['HX Mole Fraction in Top Outler Stream'][2],
        min=feature_ranges['HX Mole Fraction in Top Outler Stream'][0],
        max=feature_ranges['HX Mole Fraction in Top Outler Stream'][1],
        step=0.001,
        description='HX Mole Fraction Top:',
        style={'description_width': 'initial'}
    )
    
    widgets_dict['feed_mole_fraction'] = widgets.FloatSlider(
        value=feature_ranges['Feed Mole Fraction'][2],
        min=feature_ranges['Feed Mole Fraction'][0],
        max=feature_ranges['Feed Mole Fraction'][1],
        step=0.001,
        description='Feed Mole Fraction:',
        style={'description_width': 'initial'}
    )
    
    widgets_dict['feed_temperature'] = widgets.FloatSlider(
        value=feature_ranges['Feed Tray Temperature'][2],
        min=feature_ranges['Feed Tray Temperature'][0],
        max=feature_ranges['Feed Tray Temperature'][1],
        step=0.1,
        description='Feed Temperature:',
        style={'description_width': 'initial'}
    )
    
    widgets_dict['main_pressure'] = widgets.FloatSlider(
        value=feature_ranges['Main Tower Pressure'][2],
        min=feature_ranges['Main Tower Pressure'][0],
        max=feature_ranges['Main Tower Pressure'][1],
        step=0.1,
        description='Main Tower Pressure:',
        style={'description_width': 'initial'}
    )
    
    widgets_dict['bottom_pressure'] = widgets.FloatSlider(
        value=feature_ranges['Bottom Tower Pressure'][2],
        min=feature_ranges['Bottom Tower Pressure'][0],
        max=feature_ranges['Bottom Tower Pressure'][1],
        step=0.1,
        description='Bottom Tower Pressure:',
        style={'description_width': 'initial'}
    )
    
    widgets_dict['top_pressure'] = widgets.FloatSlider(
        value=feature_ranges['Top Tower Pressure'][2],
        min=feature_ranges['Top Tower Pressure'][0],
        max=feature_ranges['Top Tower Pressure'][1],
        step=0.1,
        description='Top Tower Pressure:',
        style={'description_width': 'initial'}
    )
    
    widgets_dict['reflux_ratio'] = widgets.FloatSlider(
        value=feature_ranges['Reflux Ratio'][2],
        min=feature_ranges['Reflux Ratio'][0],
        max=feature_ranges['Reflux Ratio'][1],
        step=0.1,
        description='Reflux Ratio:',
        style={'description_width': 'initial'}
    )
    
    # Alpha parameter for hybrid model weighting
    widgets_dict['alpha'] = widgets.FloatSlider(
        value=0.5,
        min=0.0,
        max=1.0,
        step=0.1,
        description='Alpha (FP weight):',
        style={'description_width': 'initial'}
    )
    
    return widgets_dict

# Create the widgets
input_widgets = create_input_widgets()

# Display widgets in a nice layout
left_column = widgets.VBox([
    input_widgets['liquid_condenser'],
    input_widgets['condenser_pressure'],
    input_widgets['liquid_reboiler'],
    input_widgets['feed_flow'],
    input_widgets['top_outlet_flow'],
    input_widgets['net_mass_flow'],
    input_widgets['hx_reboiler']
])

right_column = widgets.VBox([
    input_widgets['hx_top_outlet'],
    input_widgets['feed_mole_fraction'],
    input_widgets['feed_temperature'],
    input_widgets['main_pressure'],
    input_widgets['bottom_pressure'],
    input_widgets['top_pressure'],
    input_widgets['reflux_ratio']
])

control_column = widgets.VBox([
    input_widgets['alpha'],
    widgets.HTML("<br><b>Alpha = 0:</b> Pure ML model<br><b>Alpha = 1:</b> Pure First Principles<br><b>Alpha = 0.5:</b> Equal weighting")
])

input_layout = widgets.HBox([left_column, right_column, control_column])
display(input_layout)

In [ ]:
# Prediction function
def make_prediction():
    # Collect input values
    operating_conditions = {
        'Liquid Percentage in Condenser': input_widgets['liquid_condenser'].value,
        'Condenser Pressure': input_widgets['condenser_pressure'].value,
        'Liquid Percentage in Reboiler': input_widgets['liquid_reboiler'].value,
        'Mass Flow Rate in Feed Flow': input_widgets['feed_flow'].value,
        'Mass Flow Rate in Top Outlet Stream': input_widgets['top_outlet_flow'].value,
        'Net Mass Flow in main tower': input_widgets['net_mass_flow'].value,
        'Mole Fraction HX at reboiler': input_widgets['hx_reboiler'].value,
        'HX Mole Fraction in Top Outler Stream': input_widgets['hx_top_outlet'].value,
        'Feed Mole Fraction': input_widgets['feed_mole_fraction'].value,
        'Feed Tray Temperature': input_widgets['feed_temperature'].value,
        'Main Tower Pressure': input_widgets['main_pressure'].value,
        'Bottom Tower Pressure': input_widgets['bottom_pressure'].value,
        'Top Tower Pressure': input_widgets['top_pressure'].value,
        'Reflux Ratio': input_widgets['reflux_ratio'].value
    }
    
    alpha = input_widgets['alpha'].value
    
    try:
        # Make hybrid prediction
        pred_tx, pred_hx = hybrid_distillation_model(operating_conditions, alpha)
        
        # Also get individual model predictions for comparison
        rf_input = pd.DataFrame([operating_conditions])
        rf_pred = rf_model.predict(rf_input)[0]
        
        # First principles prediction
        fp_pred_tx, fp_pred_hx = first_principles_distillation_model(
            operating_conditions['Feed Mole Fraction'],
            operating_conditions['Reflux Ratio'],
            20,  # num_stages
            operating_conditions['Condenser Pressure'],
            operating_conditions['Bottom Tower Pressure'],
            operating_conditions['Feed Tray Temperature']
        )
        
        # Display results
        clear_output(wait=True)
        
        print("=" * 60)
        print("PREDICTION RESULTS")
        print("=" * 60)
        print(f"\n🔬 FIRST PRINCIPLES MODEL:")
        print(f"   MoleFractionTX: {fp_pred_tx:.6f}")
        print(f"   MoleFractionHX: {fp_pred_hx:.6f}")
        
        print(f"\n🤖 MACHINE LEARNING MODEL (Random Forest):")
        print(f"   MoleFractionTX: {rf_pred[0]:.6f}")
        print(f"   MoleFractionHX: {rf_pred[1]:.6f}")
        
        print(f"\n🔄 HYBRID MODEL (α = {alpha:.1f}):")
        print(f"   MoleFractionTX: {pred_tx:.6f}")
        print(f"   MoleFractionHX: {pred_hx:.6f}")
        
        print(f"\n📊 MODEL COMPARISON:")
        print(f"   TX Difference (FP vs ML): {abs(fp_pred_tx - rf_pred[0]):.6f}")
        print(f"   HX Difference (FP vs ML): {abs(fp_pred_hx - rf_pred[1]):.6f}")
        
        # Visualization
        fig, ax = plt.subplots(1, 2, figsize=(12, 5))
        
        models = ['First Principles', 'Random Forest', 'Hybrid']
        tx_values = [fp_pred_tx, rf_pred[0], pred_tx]
        hx_values = [fp_pred_hx, rf_pred[1], pred_hx]
        
        colors = ['blue', 'green', 'red']
        
        ax[0].bar(models, tx_values, color=colors, alpha=0.7)
        ax[0].set_title('MoleFractionTX Predictions')
        ax[0].set_ylabel('MoleFractionTX')
        ax[0].tick_params(axis='x', rotation=45)
        
        ax[1].bar(models, hx_values, color=colors, alpha=0.7)
        ax[1].set_title('MoleFractionHX Predictions')
        ax[1].set_ylabel('MoleFractionHX')
        ax[1].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error making prediction: {str(e)}")

# Create predict button
predict_button = widgets.Button(
    description='🔮 Make Prediction',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

predict_button.on_click(lambda x: make_prediction())

display(predict_button)

## 4. Model Insights and Feature Importance

In [ ]:
# Feature importance from Random Forest
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(data=feature_importance, x='Importance', y='Feature')
plt.title('Random Forest Feature Importance')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

print("Top 5 Most Important Features:")
for i, row in feature_importance.head().iterrows():
    print(f"{row['Feature']}: {row['Importance']:.4f}")

## 5. Conclusions

This hybrid model combines the theoretical foundation of first principles modeling with the data-driven accuracy of machine learning. Key benefits:

1. **Interpretability**: First principles component provides physical understanding
2. **Accuracy**: Machine learning component captures complex non-linear relationships
3. **Flexibility**: Alpha parameter allows tuning the balance between models
4. **Robustness**: Hybrid approach can handle scenarios where one model might fail

### Usage Recommendations:
- Use α closer to 1.0 when operating conditions are well within design parameters
- Use α closer to 0.0 when operating in conditions similar to historical data
- Use α = 0.5 as a balanced starting point

### Future Improvements:
- Implement more rigorous first principles equations (MESH equations)
- Add uncertainty quantification
- Include dynamic weighting based on operating conditions
- Validate against additional experimental data